In [1]:
import os
import time
import pickle

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [11]:
# 定数
main_url = "https://news.livedoor.com"
# catogories = ["main", "dom", "world", "eco", "ent", "sports", ""]
op = Options()
# --headlessだけではOSによって動かない、プロキシが弾かれる、
# CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
# 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
op.add_argument("--disable-gpu");
op.add_argument("--disable-extensions");
op.add_argument("--proxy-server='direct://'");
op.add_argument("--proxy-bypass-list=*");
op.add_argument("--start-maximized");
op.add_argument("--headless");
driver = webdriver.Chrome(options=op)
# driver = webdriver.Chrome()
driver.get(main_url)

In [12]:
# 関数
def rand_sleep():
    # 10秒〜30秒の間でランダムにスリープ
    time.sleep(np.random.randint(10, 30))

In [13]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [14]:
category_urls = []
for ele in soup.find("div", attrs={"class", "navInner"}).find_all("li"):
    category_urls.append(ele.find("a").get("href"))
# https://news.livedoor.com/は外す
category_urls.pop(0)
category_urls

['https://news.livedoor.com/topics/category/main/',
 'https://news.livedoor.com/topics/category/dom/',
 'https://news.livedoor.com/topics/category/world/',
 'https://news.livedoor.com/topics/category/eco/',
 'https://news.livedoor.com/topics/category/ent/',
 'https://news.livedoor.com/topics/category/sports/',
 'https://news.livedoor.com/article/category/52/',
 'https://news.livedoor.com/topics/category/gourmet/',
 'https://news.livedoor.com/topics/category/love/',
 'https://news.livedoor.com/topics/category/trend/',
 'https://news.livedoor.com/straight_news/',
 'https://news.livedoor.com/ranking/',
 'https://news.livedoor.com/social_reaction/',
 'https://news.livedoor.com/category/vender/special/']

In [15]:
# データを入れておき後でdataframeに変換
# すでにスクレイピング済みのデータがある場合はそっちを使う
try:
    with open('./article_infos.bin', 'rb') as f:
        article_infos = pickle.load(f)
except:
    article_infos = []
print(len(article_infos))
article_infos[-1]

1391


['https://news.livedoor.com/article/detail/20497546/',
 'None Date',
 'None Title',
 'None Text',
 'None Category',
 'None Keyword',
 '12',
 'None Vender']

In [16]:
# スクレピング開始
for category_url in category_urls[0:1]:
    # 初期変数
    # pager_idx = 1
    pager_idx = 87 # 前回87ページまで行ったため
    # 既にスクレイピング済みのサイトのURL
    scraped_page_urls = set([article[0] for article in article_infos])
    
    # ex. https://news.livedoor.com/topics/category/main/?p=1 にアクセスして解析
    category_pager_url = "{0}?p={1}".format(category_url, pager_idx)
    driver.get(category_pager_url)
    print("{}のサイトから1つずつページをスクレイピング".format(category_pager_url))
    rand_sleep()
    # 大元のソープ
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # 次のページへがなくなるまで繰り返す
    while True:
    # 実際にデータを収集するページのリンクを抜き取る
        for ele in soup.find("div", attrs={"class": "mainBody"}).find_all("li"):
            # リンクを修正する
            page_url = ele.find("a").get("href").replace("topics", "article")
            # すでにスクレイピング済みだったら
            if page_url in scraped_page_urls:
                print("{}は既にスクレイピング済です".format(page_url))
                continue
            # ex. https://news.livedoor.com/article/detail/20558207/にアクセスする
            print("{}をスクレピング開始".format(page_url))
            driver.get(page_url)
            rand_sleep()
            
            # スクレピング対象の記事
            article_soup = BeautifulSoup(driver.page_source, "html.parser")
            # データ習得
            try:
                article_category = "<SEP>".join([c.text for c in article_soup.find("nav", attrs={"class": "breadcrumbs"}).find_all("span", attrs={"itemprop": "name"})])
            except:
                article_category = "None Category"
            try:
                article_keyword = "<SEP>".join([k.text for k in article_soup.find("ul", attrs={"class": "articleHeadKeyword"}).find_all("a")])
            except:
                article_keyword = "None Keyword"
            try:
                article_date = article_soup.find("time", attrs={"class": "articleDate"}).text
            except:
                article_date = "None Date"
            try:
                article_vender = article_soup.find("p", attrs={"class": "articleVender"}).find("span", attrs={"itemprop": "name"}).text.strip(" ").strip("\n")
            except:
                article_vender = "None Vender"
            try:
                article_tweet_counts = article_soup.find("ul", attrs={"class": "socialBtn"}).find("div", attrs={"class": "tweet_counts"}).text
            except:
                article_tweet_counts
            try:
                article_title = article_soup.find("h1", attrs={"class": "articleTtl"}).text
            except:
                article_title = "None Title"
            try:
                article_text = article_soup.find("span", attrs={"itemprop": "articleBody"}).text
            except:
                article_text = "None Text"
            # データを一旦listに保存
            article_infos.append([page_url, article_date, article_title, article_text, article_category, article_keyword, article_tweet_counts, article_vender])
            print("{}の抽出完了".format(page_url))
            print("記事のタイトル: {0}, 記事の本文一部: {1}".format(article_title, article_text[0:10]))
        
        # ?p=〇ごとにデータ保存
        with open('article_infos.bin', 'wb') as f:
            print("記事数{}をpickleに保存します".format(len(article_infos)))
            pickle.dump(article_infos , f)
        # 次のページを進むのボタンがあれば
        if soup.find("ul", attrs={"class": "pager"}).find("li", attrs={"class": "next"}):
            pager_idx += 1
            category_pager_url = "{0}?p={1}".format(category_url, pager_idx)
            print("{0}ページ目の{1}に進みます".format(pager_idx, category_pager_url))
            driver.get(category_pager_url)
            rand_sleep()
            
            soup = BeautifulSoup(driver.page_source, "html.parser")
        # 最後のページのためループを抜ける
        else:
            print("次のカテゴリへ進みます")
            break
#     # 6ページ目まで行ったら一旦終了
#     if pager > 5:
#         break

https://news.livedoor.com/topics/category/main/?p=69のサイトから1つずつページをスクレイピング
https://news.livedoor.com/article/detail/20497209/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20489539/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20494829/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20495095/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20497546/は既にスクレイピング済です
https://news.livedoor.com/article/detail/20496015/をスクレピング開始
https://news.livedoor.com/article/detail/20496015/の抽出完了
記事のタイトル: 橋下徹氏、ロッキン中止なら五輪も中止にしないと「国民は言うこときかない」, 記事の本文一部: 
　元大阪市長で弁護
https://news.livedoor.com/article/detail/20497494/をスクレピング開始
https://news.livedoor.com/article/detail/20497494/の抽出完了
記事のタイトル: 秀吉の朱印状、清水寺で「再発見」　母の病気平癒の謝意, 記事の本文一部: 
　豊臣秀吉が母親の
https://news.livedoor.com/article/detail/20497448/をスクレピング開始
https://news.livedoor.com/article/detail/20497448/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20497346/をスクレピング開始
https://news.livedoor.com/article/

https://news.livedoor.com/article/detail/20495804/の抽出完了
記事のタイトル: 【大谷と一問一答】松井秀喜さんからの祝福「素直にうれしい」「まだまだ打てるよう頑張る」, 記事の本文一部: 
　エンゼルスの大谷
https://news.livedoor.com/article/detail/20495445/をスクレピング開始
https://news.livedoor.com/article/detail/20495445/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20495673/をスクレピング開始
https://news.livedoor.com/article/detail/20495673/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20495747/をスクレピング開始
https://news.livedoor.com/article/detail/20495747/の抽出完了
記事のタイトル: ＲＡＤＷＩＭＰＳ野田洋次郎「『ふざけんな』という気持ちです」, 記事の本文一部: 
RADWIMPSの
https://news.livedoor.com/article/detail/20495629/をスクレピング開始
https://news.livedoor.com/article/detail/20495629/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20495374/をスクレピング開始
https://news.livedoor.com/article/detail/20495374/の抽出完了
記事のタイトル: 新宿駅前の「巨大ネコ」街頭ビジョンが撮影スポットに？ 撮影者「時代の進歩を感じる」, 記事の本文一部: 
　新宿駅でスマート
https://news.livedoor.com/article/det

https://news.livedoor.com/article/detail/20492855/の抽出完了
記事のタイトル: 「完全に一線を越えてしまっている」　デンベレ差別発言、日本在住フランス人YouTuberが糾弾, 記事の本文一部: 
東京都在住のフラン
https://news.livedoor.com/article/detail/20493299/をスクレピング開始
https://news.livedoor.com/article/detail/20493299/の抽出完了
記事のタイトル: 小倉清一郎・横浜高元部長　松坂から「もう完全に無理です」の電話　「残念。でもよく頑張った」, 記事の本文一部: 
　西武・松坂大輔投
https://news.livedoor.com/article/detail/20493329/をスクレピング開始
https://news.livedoor.com/article/detail/20493329/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20493225/をスクレピング開始
https://news.livedoor.com/article/detail/20493225/の抽出完了
記事のタイトル: 熱海の盛り土に産廃、再三指導　土石流起点、計画超の高さ50m, 記事の本文一部: 
大規模土石流の起点
https://news.livedoor.com/article/detail/20493245/をスクレピング開始
https://news.livedoor.com/article/detail/20493245/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
記事数1486をpickleに保存します
74ページ目のhttps://news.livedoor.com/topics/category/main/?p=74に進みます
https://news.livedoor.com/article/detail/20493245/をスクレピング開始
https://news.livedoor.com/article/detai

https://news.livedoor.com/article/detail/20490372/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20490381/をスクレピング開始
https://news.livedoor.com/article/detail/20490381/の抽出完了
記事のタイトル: コロチキナダル「スタッフより演者の方が上！」業界評価最下位でブチギレ, 記事の本文一部: 
　コロコロチキチキ
https://news.livedoor.com/article/detail/20489065/をスクレピング開始
https://news.livedoor.com/article/detail/20489065/の抽出完了
記事のタイトル: 「幼稚園の母娘」に襲いかかり…飢えた北朝鮮兵士が凶悪化, 記事の本文一部: 
北朝鮮と中国の間を
https://news.livedoor.com/article/detail/20490295/をスクレピング開始
https://news.livedoor.com/article/detail/20490295/の抽出完了
記事のタイトル: 【MLB】大谷翔平、相手打者の折れたバット拾う“神行動”　米記者も感嘆「紳士だ」「とても素敵」, 記事の本文一部: 
2回まで1失点の大
https://news.livedoor.com/article/detail/20490576/をスクレピング開始
https://news.livedoor.com/article/detail/20490576/の抽出完了
記事のタイトル: 人種差別問題でグリーズマンが楽天の三木谷会長に謝罪, 記事の本文一部: 
　日本人に対する人
https://news.livedoor.com/article/detail/20488057/をスクレピング開始
https://news.livedoor.com/article/detail/20488057/の抽出完了
記事のタイトル: 【海外発！Breaking News】「ド派手にやって」まつエクサロンが客の要望に応えた結果、毛虫のように（米）, 

https://news.livedoor.com/article/detail/20488162/の抽出完了
記事のタイトル: 有吉弘行　結婚で「仕事セーブ」でも見えてきた“島田紳助超え”の野心, 記事の本文一部: 
　4月にフリーアナ
https://news.livedoor.com/article/detail/20489359/をスクレピング開始
https://news.livedoor.com/article/detail/20489359/の抽出完了
記事のタイトル: 西武、松坂の引退を正式発表　渡辺GM「体調面、精神面で万全とは言えない状況…回復した段階で会見を」, 記事の本文一部: 
　西武が7日、松坂
https://news.livedoor.com/article/detail/20488896/をスクレピング開始
https://news.livedoor.com/article/detail/20488896/の抽出完了
記事のタイトル: 【海外発！Breaking News】マクドナルド従業員がシフト中に一斉退職　「最悪の労働環境」と訴え（米）, 記事の本文一部: 
先月28日、マクド
https://news.livedoor.com/article/detail/20489100/をスクレピング開始
https://news.livedoor.com/article/detail/20489100/の抽出完了
記事のタイトル: 熊田曜子、大誤算で夫とは和解の方向へ？ 双方一歩も譲れずドロ沼化しそうな問題も, 記事の本文一部: 
　自身への暴行容疑
https://news.livedoor.com/article/detail/20488799/をスクレピング開始
https://news.livedoor.com/article/detail/20488799/の抽出完了
記事のタイトル: 豊洲市場付近のビアガーデンに “ノーマスク外国人” 殺到…五輪関係者も癒やしの場に, 記事の本文一部: 
「選手村の対岸、豊
https://news.livedoor.com/article/detail/20489309/をスクレピング開始
https://news.livedoor.com/article/detail/20489309

https://news.livedoor.com/article/detail/20486748/の抽出完了
記事のタイトル: 天皇陛下がワクチン接種, 記事の本文一部: 
　天皇陛下は6日、
https://news.livedoor.com/article/detail/20484918/をスクレピング開始
https://news.livedoor.com/article/detail/20484918/の抽出完了
記事のタイトル: 田中みな実に瞬殺で断られ…TBS安住アナ新番組の“相棒”探しが迷走中, 記事の本文一部: 
　TBS系で10月
https://news.livedoor.com/article/detail/20486125/をスクレピング開始
https://news.livedoor.com/article/detail/20486125/の抽出完了
記事のタイトル: 安藤なつ、夫だけでなく元相方にも行っていた“24時間監視”と“モラハラ”, 記事の本文一部: 
　6月末、突如“離
https://news.livedoor.com/article/detail/20486571/をスクレピング開始
https://news.livedoor.com/article/detail/20486571/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20486366/をスクレピング開始
https://news.livedoor.com/article/detail/20486366/の抽出完了
記事のタイトル: 上沼恵美子　冠番組終了「ものすごい失礼な局の対応に腹立って」９月までやらずと暴露, 記事の本文一部: 
　タレントの上沼恵
https://news.livedoor.com/article/detail/20486287/をスクレピング開始
https://news.livedoor.com/article/detail/20486287/の抽出完了
記事のタイトル: 「現場は大混乱」「制度設計に甘さ」福岡知事が国に苦言, 記事の本文一部: 
　新型コロナウイル
https://news.livedoor.

https://news.livedoor.com/article/detail/20484512/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
記事数1666をpickleに保存します
83ページ目のhttps://news.livedoor.com/topics/category/main/?p=83に進みます
https://news.livedoor.com/article/detail/20484512/をスクレピング開始
https://news.livedoor.com/article/detail/20484512/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20484354/をスクレピング開始
https://news.livedoor.com/article/detail/20484354/の抽出完了
記事のタイトル: 大谷翔平のバットが真っ二つに！ファンも「この折れ方、見たことない」「裂けた！」とどよめき, 記事の本文一部: 
【米大リーグ】エン
https://news.livedoor.com/article/detail/20484410/をスクレピング開始
https://news.livedoor.com/article/detail/20484410/の抽出完了
記事のタイトル: 「ヲタクが気持ち悪い」と本音ぶっちゃけ　話題の地下アイドルがグループ脱退へ, 記事の本文一部: 
広島県を中心に活動
https://news.livedoor.com/article/detail/20483946/をスクレピング開始
https://news.livedoor.com/article/detail/20483946/の抽出完了
記事のタイトル: 被買収100人「異例」の不起訴へ　河井夫妻の選挙買収, 記事の本文一部: 
　2019年7月の
https://news.livedoor.com/article/detail/20484427/をスクレピング開始
https://news.livedoor.com/article/detail/20484427/の抽出完

https://news.livedoor.com/article/detail/20482827/の抽出完了
記事のタイトル: デンベレ、日本人への差別発言を謝罪も再炎上…グリーズマンは別の差別疑惑が浮上, 記事の本文一部: 
バルセロナに所属す
https://news.livedoor.com/article/detail/20482634/をスクレピング開始
https://news.livedoor.com/article/detail/20482634/の抽出完了
記事のタイトル: 深夜宴会YouTuberえびすじゃっぷ、発言謝罪も...　「言い訳」続けて批判再燃, 記事の本文一部: 
緊急事態宣言下で行
https://news.livedoor.com/article/detail/20482376/をスクレピング開始
https://news.livedoor.com/article/detail/20482376/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20482382/をスクレピング開始
https://news.livedoor.com/article/detail/20482382/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/article/detail/20482467/をスクレピング開始
https://news.livedoor.com/article/detail/20482467/の抽出完了
記事のタイトル: デンベレ＆グリーズマンが侮辱発言を釈明し謝罪「日本の友人たちを怒らせたなら申し訳ない」, 記事の本文一部: 
　バルセロナに所属
https://news.livedoor.com/article/detail/20482737/をスクレピング開始
https://news.livedoor.com/article/detail/20482737/の抽出完了
記事のタイトル: None Title, 記事の本文一部: None Text
https://news.livedoor.com/art

KeyboardInterrupt: 